In [1]:
#import dependencies
import pandas as pd
import numpy as np
import sqlalchemy as db
import config
import requests
import csv
import googlemaps
import datetime


from pathlib import Path
from dateutil.parser import parse
from datetime import date
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, Integer, Table, Column, MetaData
from sqlalchemy.orm import Session

In [2]:
# Create SQLite db
restaurants_db = "restaurants.sqlite"
connector = (f"sqlite:///{restaurants_db}")

# Create engine
engine = create_engine(connector)

# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

# Print all of the classes (tables) mapped to the Base
Base.classes.keys()

# Create the inspector and connect it to the engine
inspector = inspect(engine)

In [3]:
def verify_inserts(tablename):
    # Using the inspector to print the column names within the 'restaurants' table and its types
    columns = inspector.get_columns(tablename)
    for column in columns:
        print(column["name"], column["type"])

    # Create metadata access to the object
    meta_data = db.MetaData(bind=engine)
    db.MetaData.reflect(meta_data)

    # Get the table from the metadata object
    restaurants = meta_data.tables['Restaurants']

    # Select and print rowcount
    result = db.select([db.func.count()]).select_from(restaurants).scalar()
    print("Row Count:", result)

In [4]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

In [5]:
def convert_to_date(date_string):
    date = parse("2015-02-24T13:00:00-08:00")
    
    return date

In [6]:
def add_change_date(df):
    df["change_date"] = date.today()
    
    return df

In [7]:
def str_to_bool(df, col):
    df.notnull()
    return col.map({'True': True, 1: True, 0: False, 'False': False, 'Na':False,})

## Load and clean yelp_business.csv

In [8]:
def etl_restaurants():
    # Load business into df
    business_path = Path('../data_zipped/yelp_business.csv.zip', low_memory=False)
    business_df = pd.read_csv(business_path)

    # Filter businesses to restaurants only
    business_df = business_df[business_df["categories"].str.contains("Restaurant")]
    
    # Drop rows where is_open = 0
    business_df = business_df[business_df.is_open == 0]

    # Add flag "is_mexican_restaurant"
    business_df["is_mexican_restaurant"] = np.where(business_df.categories.str.contains("Mex"),1,0)

    # Remove is_open, neighborhood, and categories
    business_df = business_df.drop(['is_open', 'neighborhood', 'categories'], axis=1)

    # Fill missing values with empty string
    business_df = business_df.fillna('')

    # Add change date
    add_change_date(business_df)
    
    # Write to SQLite db
    business_df.to_sql('Restaurants', engine, if_exists='replace')
    
    return business_df


## Load and clean Reviews

In [9]:
def etl_reviews(df):
# Load comments into df
    reviews_path = Path('../data_zipped/yelp_tip.csv.zip', low_memory=False)
    reviews_df = pd.read_csv(reviews_path)

    # Drop rows with missing values
    reviews_df = reviews_df.dropna(axis=0,how="any")

    # Merge with business_df to filter data set to relavent rows
    reviews_df = business_df.set_index('business_id').join(reviews_df.set_index('business_id'), rsuffix='_review')

    # Eliminate unneeded columns
    reviews_df = reviews_df.loc[:,"text":"user_id"]
    
    # Name dataframe index and write to table
    reviews_df['review_id'] = reviews_df.index
    
    # Reorder columns so key is in the first position
    reviews_df.insert(0, 'review_id', reviews_df.pop('review_id'))
    
    # Convert "date" column to a date datatype
    reviews_df["date"] = convert_to_date(reviews_df["date"])
    
    # Add ChangeDate
    add_change_date(reviews_df)
    
    # Write to database
    reviews_df.to_sql('Reviews', engine, if_exists='replace')

    # All done, release memory
    del reviews_df

## Load and clean yelp_business_attributes.csv

In [10]:
def etl_restaurant_attributes(df):
    # Load business attributes into df
    attributes_path = Path('../data_zipped/yelp_business_attributes.csv.zip', low_memory=False)
    attributes_df = pd.read_csv(attributes_path)

    # Rearrange column names (not columns) due to defect in source data set
    col_list = list(attributes_df)
    col_list.insert(2, col_list.pop())
    attributes_df.columns = col_list

    # Merge with business_df to filter data set to relavent rows
    restaurant_attribute_df = business_df.set_index('business_id').join(attributes_df.set_index('business_id'), rsuffix='_attr')

    # Release memory
    del attributes_df

    # List of category columns
        # Column: Alcohol, Values: ['Na' 'full_bar' 'beer_and_wine' 'none']
        # Column: NoiseLevel, Values: ['Na' 'average' 'very_loud' 'quiet' 'loud']
        # Column: RestaurantsAttire, Values: ['Na' 'casual']
        # Column: WiFi, Values: ['Na' 'free' 'paid' 'no']
        # Column: Smoking, Values: ['Na' 'no' 'outdoor' 'yes']
        # Column: BYOBCorkage, Values: ['Na' 'yes_free' 'no' 'yes_corkage']
    string_cols = ["Alcohol","NoiseLevel","RestaurantsAttire","WiFi","Smoking","BYOBCorkage"]

    # For column in the dataframe:
    for col in col_list[1:]:
        # Skip category columns for string conversion
        if col not in string_cols:
            # Convert to boolean
            if(restaurant_attribute_df.loc[:,col].nunique()) <= 3:
                restaurant_attribute_df[col] = str_to_bool(restaurant_attribute_df,restaurant_attribute_df[col])

        # Eliminate columns with only one distinct value
        if(restaurant_attribute_df.loc[:,col].nunique()) == 1:
            restaurant_attribute_df.pop(col)

    # Convert 'Na' to 0 and then convert col to int
    # Column: RestaurantsPriceRange2, Values: ['Na' '1' '3' '2' '4']
    restaurant_attribute_df["RestaurantsPriceRange2"] = restaurant_attribute_df["RestaurantsPriceRange2"].replace(to_replace='Na', value=0)

    # Convert remaining "Na" to "no", affects these columns:
    restaurant_attribute_df[string_cols] = restaurant_attribute_df[string_cols].replace(to_replace='Na', value='no')

    # Send entire business_df to csv for ML; ML workflow will take care of encoding and imputation from here
    restaurant_attribute_df.to_csv('../data_zipped/restaurants.csv')

    # Eliminate business_df elements
    attribute_df = restaurant_attribute_df.loc[:,"change_date":"DietaryRestrictions_vegetarian"]

    # Drop rows without attributes
    attribute_df = attribute_df.dropna(axis=0,how="any")

    # Write to SQLite db
    attribute_df.to_sql('RestaurantAttributes', engine, if_exists='replace')

## Clean and load business data for mexican_restaurants.csv

In [11]:
def etl_mex_restaurants(df, gmaps_key, dbg):
    # Limit to Mexican restaurants only as that is our dashboard focus
    # Saves about 12,000 API calls and probably a few dineros!

    # Create a formatted address string for geocoding
    mex_restaurants_df['full_address'] = mex_restaurants_df[['address', 'city', "state",'postal_code']].agg(', '.join, axis=1).str.replace('"','')

    # Run the address through the geocoder function and apply the results to a new column called "geocoded"
    mex_restaurants_df['geocoded'] = mex_restaurants_df['full_address'].apply(geocode)

    # # Apply the new geocodes to the restaurants dataframe
    mex_restaurants_df[['latitude', 'longitude']] = pd.DataFrame(mex_restaurants_df['geocoded'].tolist(), index=df.index)

    # If not a debug run save the results to a csv file for BI
    if dbg == "N": 
        mex_restaurants_df.to_csv("mexican_restaurants.csv")
        
    return mex_restaurants_df.head(100)

In [12]:
# Run Restaurants ETL
business_df = etl_restaurants()

# Validate success
verify_inserts('Restaurants')

index BIGINT
business_id TEXT
name TEXT
address TEXT
city TEXT
state TEXT
postal_code TEXT
latitude FLOAT
longitude FLOAT
stars FLOAT
review_count BIGINT
is_mexican_restaurant INTEGER
change_date DATE
Row Count: 14225


In [13]:
# Run Reviews ETL
etl_reviews(business_df)

# Validate success
verify_inserts('Reviews')

business_id TEXT
review_id TEXT
text TEXT
date TIMESTAMP
likes FLOAT
user_id TEXT
change_date DATE
Row Count: 14225


In [14]:
# Run RestaurantAttributes ETL
etl_restaurant_attributes(business_df)

# Validate success
verify_inserts('RestaurantAttributes')

business_id TEXT
change_date DATE
ByAppointmentOnly BOOLEAN
BusinessAcceptsCreditCards BOOLEAN
BusinessParking_garage BOOLEAN
BusinessParking_street BOOLEAN
BusinessParking_validated BOOLEAN
BusinessParking_lot BOOLEAN
BusinessParking_valet BOOLEAN
RestaurantsPriceRange2 TEXT
GoodForKids BOOLEAN
WheelchairAccessible BOOLEAN
BikeParking BOOLEAN
Alcohol TEXT
HasTV BOOLEAN
NoiseLevel TEXT
RestaurantsAttire TEXT
Music_dj BOOLEAN
Music_karaoke BOOLEAN
RestaurantsGoodForGroups BOOLEAN
Caters BOOLEAN
WiFi TEXT
RestaurantsReservations BOOLEAN
RestaurantsTakeOut BOOLEAN
HappyHour BOOLEAN
GoodForDancing BOOLEAN
RestaurantsTableService BOOLEAN
OutdoorSeating BOOLEAN
RestaurantsDelivery BOOLEAN
BestNights_monday BOOLEAN
BestNights_friday BOOLEAN
BestNights_wednesday BOOLEAN
BestNights_thursday BOOLEAN
BestNights_sunday BOOLEAN
BestNights_saturday BOOLEAN
GoodForMeal_dessert BOOLEAN
GoodForMeal_latenight BOOLEAN
GoodForMeal_lunch BOOLEAN
GoodForMeal_dinner BOOLEAN
GoodForMeal_breakfast BOOLEAN
Good

In [15]:
# Dbg flag indicates if a debug execution or not, and will conditionally skip certain tasks both in this cell and within the geocode function
# This is so that the code can be executed in "demo" mode and will reduce Google API calls to avoid charges
dbg = "N"

# Set API key from config file
gmaps_key = googlemaps.Client(key=config.API_KEY)

# Filter input df to only geocode Mexican Restaurants to save on API calls
mex_restaurants_df = business_df[business_df["is_mexican_restaurant"] == 1]

# For "demo" mode only
if dbg == 'Y':
    mex_restaurants_df = mex_restaurants_df.head()

etl_mex_restaurants(mex_restaurants_df, gmaps_key, dbg)

C:\Users\curt0\AppData\Local\Temp/ipykernel_23700/2902016065.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mex_restaurants_df['full_address'] = mex_restaurants_df[['address', 'city', "state",'postal_code']].agg(', '.join, axis=1).str.replace('"','')
C:\Users\curt0\AppData\Local\Temp/ipykernel_23700/2902016065.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mex_restaurants_df['geocoded'] = mex_restaurants_df['full_address'].apply(geocode)
C:\Users\curt0\anaconda3\lib\site-packages\pandas\core\frame

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_mexican_restaurant,change_date,full_address,geocoded
313,WEouNHHxfqGa8gYAnyiyBA,"""El Palenque""","""5945 Andrews Rd""",Mentor-on-the-Lake,OH,44060,41.709322,-81.359461,3.5,3,1,2022-08-11,"5945 Andrews Rd, Mentor-on-the-Lake, OH, 44060","(41.7093216, -81.35946129999999)"
478,to2cGMKdaw7ZHbXMzpfhlA,"""Filiberto's Mexican Rest""","""3218 E Mcdowell Rd""",Phoenix,AZ,85008,33.466004,-112.012286,3.0,7,1,2022-08-11,"3218 E Mcdowell Rd, Phoenix, AZ, 85008","(33.46600430000001, -112.0122861)"
868,kiU9C58n7dgygdrpjOQXMQ,"""Los Jarochos Restaurant""","""4811 S Rainbow Blvd""",Las Vegas,NV,89147,36.101020,-115.244312,4.0,62,1,2022-08-11,"4811 S Rainbow Blvd, Las Vegas, NV, 89147","(36.1010205, -115.2443122)"
951,Ka3z0iudeviL2nEKZ4BeYg,"""Tijuana Flats""","""1608 E Blvd""",Charlotte,NC,28203,35.199318,-80.841358,4.0,50,1,2022-08-11,"1608 E Blvd, Charlotte, NC, 28203","(35.1993181, -80.8413576)"
962,v0KawuAO7clhXCDKobFs0g,"""Currito - Burritos Without Borders""","""Pittsburgh International Airport""",Pittsburgh,PA,15231,40.495309,-80.235291,3.0,71,1,2022-08-11,"Pittsburgh International Airport, Pittsburgh, ...","(40.495309, -80.2352913)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16224,Q28MO9hEMyG0xrAnOF-dSw,"""Stand Up Scottsdale Comedy Club""","""5101 N Scottsdale Rd""",Scottsdale,AZ,85251,33.510801,-111.925514,4.5,190,1,2022-08-11,"5101 N Scottsdale Rd, Scottsdale, AZ, 85251","(33.5108012, -111.9255139)"
16228,5UaVi_cwnoAvIBp0g4aF2Q,"""Birria Estilo Michoacan""","""906 N 15th Ave""",Phoenix,AZ,85007,33.457929,-112.091487,4.0,5,1,2022-08-11,"906 N 15th Ave, Phoenix, AZ, 85007","(33.4579287, -112.0914873)"
16330,5KWJZHhG88echvquDBu4iw,"""Tronco Sonoran Grill""","""10155 E Via Linda, Unit 135""",Scottsdale,AZ,85258,33.574430,-111.862215,3.0,45,1,2022-08-11,"10155 E Via Linda, Unit 135, Scottsdale, AZ, 8...","(33.5744296, -111.862215)"
16377,zY_YAwo4On0aqOABS-1cBA,"""Rockin' Taco Mexican Grill""","""3717 S Las Vegas Blvd, Ste 260""",Las Vegas,NV,89109,36.108842,-115.172149,3.5,63,1,2022-08-11,"3717 S Las Vegas Blvd, Ste 260, Las Vegas, NV,...","(36.1088418, -115.1721488)"


In [16]:
# classical supervised
# predict stars given business attributes
# get probability then multiply by stars

# natural language
# predict review rating based on text
# KNN model